<a href="https://colab.research.google.com/github/kasparvonbeelen/heritageweaver/blob/2-vdb/colab_notebooks/Fine_tune_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from transformers import SiglipProcessor, SiglipModel

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!gdown 1mIbvagKtuZY7pBkHipKe2oK7lfrBrdak
!unzip -q -o img_data

Downloading...
From (original): https://drive.google.com/uc?id=1mIbvagKtuZY7pBkHipKe2oK7lfrBrdak
From (redirected): https://drive.google.com/uc?id=1mIbvagKtuZY7pBkHipKe2oK7lfrBrdak&confirm=t&uuid=238a0c22-77c1-4c1e-acdb-b8b416db11ab
To: /content/img_data.zip
100% 1.51G/1.51G [00:09<00:00, 154MB/s]


In [ ]:
# Create folder for saving the model
Path('models').mkdir(exist_ok=True)

In [ ]:
# Choose computation device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
# Load SigLIP model and processor
checkpoint = "google/siglip-base-patch16-224" # "Kaspar/siglip-heritage-weaver-name"
processor = SiglipProcessor.from_pretrained(checkpoint)
model = SiglipModel.from_pretrained(checkpoint)
model.to(device)

print('imported everything :-)')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

imported everything :-)


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/kasparvonbeelen/heritageweaver/2-vdb/data/heritage_weaver_data.csv', index_col=0)
df.shape

(31153, 11)

In [ ]:
#df.loc[:,'name'].fillna('', inplace=True)
df.fillna({'name':''}, inplace=True)
df = df[['record_id','name','description','img_path']].dropna()

df['filepath'] = df['img_path'].apply(lambda x: '/content/images/' + x.split('/')[-1])
df['downloaded'] =df['filepath'].apply(lambda x: Path(x).is_file())
df = df[df.downloaded==True].sample(frac=1.0, random_state=1984).reset_index(drop=True)
df.shape

(31153, 6)

In [ ]:
list(df.record_id.unique())[:5]

['co8398837', 'co8062331', 'co5427', 'co8415537', 'co22682']

In [ ]:
ids = list(df.record_id.unique())
threshold = int(len(ids)*.9)
train_ids = ids[:threshold]
eval_ids = ids[threshold:]
len(train_ids), len(eval_ids)


(19545, 2172)

## process data for training both names and descriptions

we need to chunk the descriptions into units of 64 tokens

In [ ]:
from transformers import SiglipTokenizer
tokenizer = SiglipTokenizer.from_pretrained("google/siglip-base-patch16-224")

In [ ]:
#help(tokenizer)

In [ ]:
tokenized_descriptions = [tokenizer(
                                    text,
                                    padding=False,
                                    #truncation=True,
                                    #max_length=64,
                                    #return_overflowing_tokens=True,
                                    return_tensors="pt"
                                        ) for text in df['description'].tolist()]

Token indices sequence length is longer than the specified maximum sequence length for this model (312 > 64). Running this sequence through the model will result in indexing errors


In [ ]:
#len(tokenized_descriptions[100]['input_ids'][0])
# set([len(s['input_ids'][0]) for s in tokenized_descriptions])

In [ ]:
len(tokenized_descriptions[14]['input_ids'][0])

17

In [ ]:
import re
tokenized_segments = []
for e in tokenized_descriptions:
  segments = []
  for i in range(0,len(e['input_ids'][0]),64):
    segment = e['input_ids'][0][i:min(i+64,len(e['input_ids'][0]))]

    segments.append(re.sub(r'<[^>]+>', '',tokenizer.decode(segment)))
  tokenized_segments.append(segments)


In [ ]:
df['segments'] = tokenized_segments
df['text'] = df.apply(lambda x: [x['name']] + x['segments'], axis=1)

In [ ]:
df[['name','description','text']].iloc[3].text

['pipe',
 'length of ceramic pipe length of ceramic pipe used for cable conduit to allow cables to enter through masonry house walls with bell drip ends part of the bob gray collection of electrical objects']

In [ ]:
df = df.explode('text')
df.drop_duplicates(subset=['text','img_path'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(75378, 8)

In [ ]:
df.text[:10]

,text
0,lens
1,lens for narrowing light beam for thomson mirr...
2,on mirror speaking galvanometer made by elliot...
3,sensitive electrical indicating instrument and...
4,card some distance away thus magnifying any mo...
5,to write the message down this problem was sol...
6,cooking tool and equipment
7,iron ratchet pot hook from sussex britain 1700...
8,pot hook from 16th century onwards
9,photomultiplier tube


In [ ]:

df_train = df.loc[df.record_id.isin(train_ids)]
df_eval = df.loc[df.record_id.isin(eval_ids)]
df_train[['filepath','text']].to_csv('train.csv', sep='\t') # name | description
df_eval[['filepath','text']].to_csv('eval.csv', sep='\t') # name | description


In [ ]:
df_train.shape, df_eval.shape

((70771, 8), (4607, 8))

In [ ]:
# Define a custom dataset
class image_title_dataset():
    def __init__(self, df, column='text'):  # description | name
        # Initialize data
        self.df = df
        self.column = column

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Preprocess image using SigLIPS's preprocessing function
        processed = processor(text=[self.df.iloc[idx][self.column]],
                              images=[Image.open(self.df.iloc[idx].filepath).convert('RGB')],
                         return_tensors="pt",
                         max_length=64,
                         padding='max_length', truncation=True)
        return processed['input_ids'].to(device), processed['pixel_values'].squeeze(0).to(device) #

# handy utility at https://github.com/wenwei202/pytorch-examples/blob/ecbb7beb0fac13133c0b09ef980caf002969d315/imagenet/main.py#L296

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
dataset_train = image_title_dataset(df_train)
dataset_eval = image_title_dataset(df_eval)

In [ ]:

# Prepare the optimizer
# betas=(0.9,0.98),eps=1e-6,weight_decay=0.2
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5,) # the lr is smaller, more safe for fine tuning to new dataset
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) # the lr is smaller, more safe for fine tuning to new dataset

#optimizer = torch.optim.Adam(list(model.parameters()) + list(loss_fn.parameters()), lr=.5)
train_dataloader = DataLoader(dataset_train, batch_size=16, shuffle=True) #Define your own dataloader
eval_dataloader = DataLoader(dataset_eval, batch_size=16, shuffle=True) #Define your own dataloader



In [ ]:
# # Specify the loss function
# loss_img = nn.CrossEntropyLoss()
# loss_txt = nn.CrossEntropyLoss()

loss_img = nn.BCEWithLogitsLoss()
loss_txt = nn.BCEWithLogitsLoss()

In [ ]:
# Train the model
from collections import defaultdict
num_epochs = 5

best_loss = 100000000
tracker = []

losses_train = AverageMeter()
losses_eval = AverageMeter()

for epoch in range(num_epochs):

    model.train()
    losses_train.reset()
    losses_eval.reset()

    for idx, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()

        texts, images = batch

        # Forward pass
        output = model(texts, images)

        # Compute the loss with Sigmoid
        n = output.logits_per_image.size(0)  # Batch size
        ground_truth = torch.eye(n, device=output.logits_per_image.device)  # Diagonal labels for matching pairs (positive)
        total_loss = (loss_img(output.logits_per_image,ground_truth) + loss_txt(output.logits_per_text,ground_truth))/2

        losses_train.update(total_loss.item(), len(images))

        # Backward pass
        total_loss.backward()

        optimizer.step()
        tracker.append(['train',epoch, idx, total_loss.item(),losses_train.val,losses_train.avg])
        if idx % 50 == 0:
          print('Epoch: [{0}]\t'
                'Training Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                  epoch, loss=losses_train,))

    #model.save_pretrained(f'./models/{checkpoint}-ft-name')
    model.push_to_hub("Kaspar/siglip-heritage-weaver-last")

    model.eval()

    for idx, batch in enumerate(tqdm(eval_dataloader)):
      texts,images = batch
      # Forward pass
      output = model(texts, images)
      # Compute loss
      # Compute the loss with Sigmoid
      n = output.logits_per_image.size(0)  # Batch size
      ground_truth = torch.eye(n, device=output.logits_per_image.device)  # Diagonal labels for matching pairs (positive)
      total_loss = (loss_img(output.logits_per_image,ground_truth) + loss_txt(output.logits_per_text,ground_truth))/2
      losses_eval.update(total_loss.item(), len(images))
      tracker.append(['eval',epoch, idx, total_loss.item(),losses_eval.val,losses_eval.avg])

      if idx % 50 == 0:
        print('Epoch: [{0}]\t'
                  'Eval Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                   epoch, loss=losses_eval))

    if losses_eval.avg <= best_loss:
        best_loss = losses_eval.avg
        print(f'saving best model with loss={losses_eval.avg}')
        model.push_to_hub("Kaspar/siglip-heritage-weaver-best")

processor.push_to_hub("Kaspar/siglip-heritage-weaver-last")
processor.push_to_hub("Kaspar/siglip-heritage-weaver-best")
df_tracking = pd.DataFrame(tracker, columns=['split','epoch','batch','loss','loss_val','loss_avg'])
df_tracking.to_csv('tracking.csv')

  0%|          | 1/4424 [00:00<1:07:04,  1.10it/s]

Epoch: [0]	Training Loss 0.1033 (0.1033)	


  1%|          | 51/4424 [00:46<1:07:46,  1.08it/s]

Epoch: [0]	Training Loss 0.0949 (0.1268)	


  2%|▏         | 101/4424 [01:31<1:05:35,  1.10it/s]

...

 60%|█████▉    | 2651/4424 [40:12<27:51,  1.06it/s]

Epoch: [3]	Training Loss 0.0095 (0.0311)	


 61%|██████    | 2701/4424 [40:58<25:27,  1.13it/s]

Epoch: [3]	Training Loss 0.0388 (0.0310)	


 62%|██████▏   | 2733/4424 [41:28<26:01,  1.08it/s]

In [ ]:
model.push_to_hub("Kaspar/siglip-heritage-weaver-last")

NameError: name 'model' is not defined

# Evaluation

Split train - test
- based on random split (within collection)
- based on object ids (out of sample)

Retrieval: Use keyword to query image
 - compute similarity between keyword and names of retrieved images

Zero-shot
 - zero shot classification with names/multiclass (maybe simplify names)

Retrieval image-description
(maybe blocked by keyword, i.e. within telephones)
 - query based on description, compute rank of correct image
 - query based on images, compute rank of correct description

Cluster
- Correspondence cluster with image embeddings and name labels